In [1]:
using Pkg

# LogRoller.jl: ローテーション可能なロギングを可能にするパッケージ
Pkg.add("LogRoller")

# HTTP.jl: 最新の master ブランチからインストール
Pkg.add(PackageSpec(name="HTTP", rev="master"))

    Updating registry at `C:\Users\user\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\user\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\user\.julia\environments\v1.7\Manifest.toml`
    Updating git-repo `https://github.com/JuliaWeb/HTTP.jl.git`
   Resolving package versions...
  No Changes to `C:\Users\user\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\user\.julia\environments\v1.7\Manifest.toml`


In [1]:
using HTTP, LogRoller, Logging, Sockets

"""
Global Logger: RollingLogger
- file name: "./WebSocket.log"
- size limit: 1MB (1,000,000 bytes)
- maintain rotated file count: 3
- log level: Debug
"""
# Set global logger.
global_logger(
    RollingLogger("./WebSocket.log", 1_000_000, 3, Logging.Debug)
)

"""
    createWebSocketServer(; hostname::String = "127.0.0.1", port::UInt16 = UInt16(8081))
        -> (server::IOServer, task::Task)

Create WebSocket Server.
"""
createWebSocketServer(; hostname::String = "127.0.0.1", port::UInt16 = UInt16(8081)) = begin
    server = Sockets.listen(Sockets.InetAddr(parse(IPAddr, hostname), port))
    task = @async HTTP.WebSockets.listen(hostname, port; server = server) do ws
        # タスク内のエラー処理は明示的に try-catch した方が良い
        try
            for msg in ws
                HTTP.WebSockets.send(ws, msg)
            end
        catch e
            @error "WebSocketServer Error" exception=(e, catch_backtrace())
            rethrow() # プロセスを終了するために rethrow する
        end
    end
    server, task
end

"""
    createWebSocketClient(endpoint::String)
        -> (task::Task, in_channel::Channel, out_channel::Channel)

Create WebSocket Client.

- channels:
    - in_channel:
        - type: `String`
        - size: 1
    - out_channel:
        - type `Any`
        - size: 1
- events:
    - in_channel: `exit`
        - Terminate the websocket client.
    - in_channel: `send:{any_string_data}`
        - Send `any_string_data` to the websocket server.
        - Response data will be written in out_channel: `Response: {any_string_data}`
    - in_channel: `exec:{function_name}`
        - Execute function to websocket stream: `function_name(ws::HTTP.Stream)`
        - Function return value will be written in out_channel.
"""
createWebSocketClient(endpoint::String) = begin
    c_in = Channel{String}(1)
    c_out = Channel(1)
    task = @async HTTP.WebSockets.open(endpoint) do ws
        # タスク内のエラー処理は明示的に try-catch した方が良い
        try
            # WebSocket の疎通が続く限りループする
            ## https://juliaweb.github.io/HTTP.jl/dev/reference/#HTTP.WebSockets.isclosed
            while !HTTP.WebSockets.isclosed(ws)
                # WebSocket PING 疎通確認
                try
                    HTTP.WebSockets.ping(ws)
                catch
                    break
                end

                command = take!(c_in) # wait until the in_channel is entered.
                commands = string.(split(command, ":", keepempty=false))

                @debug commands

                if commands[1] === "exit"
                    @info "Terminate websocket client..."
                    break
                elseif commands[1] === "send"
                    HTTP.WebSockets.send(ws, commands[2])
                    response = HTTP.WebSockets.receive(ws)
                    put!(c_out, string("Response: ", response))
                elseif commands[1] === "exec"
                    f = Symbol(commands[2])
                    output = @eval $f($ws)
                    @debug output
                    put!(c_out, output)
                end
            end
        catch e
            @error "WebSocketClient Error" exception=(e, catch_backtrace())
            showerror(stderr, e, catch_backtrace()) # コンソールにもエラー表示する
            rethrow() # プロセスを終了するために rethrow する
        end
    end
    task, c_in, c_out
end

createWebSocketClient

In [2]:
server, server_task = createWebSocketServer()

(Sockets.TCPServer(Base.Libc.WindowsRawSocket(0x0000000000000410) active), Task (runnable) @0x000000000de5e100)

In [3]:
client, c_in, c_out = createWebSocketClient("ws://127.0.0.1:8081")

(Task (runnable) @0x000000000f2b1180, Channel{String}(1), Channel{Any}(1))

In [4]:
istaskdone(server_task), istaskdone(client)

(false, false)

In [5]:
# Send data ("Hello WebSocket!!") to the WebSocket client.
put!(c_in, "send:Hello WebSocket!!")

"send:Hello WebSocket!!"

In [6]:
take!(c_out)

"Response: Hello WebSocket!!"

In [7]:
# Close WebSocket server.
close(server)

istaskdone(server_task), istaskdone(client)

(true, false)

In [8]:
# すでに閉じている WebSocket サーバにデータを送信した場合
put!(c_in, "send:Hello WebSocket!!")

"send:Hello WebSocket!!"

In [9]:
take!(c_out)

"Response: Hello WebSocket!!"

確認した感じ、WebSocket サーバが閉じても、クライアント側も同時に閉じるということはなさそう

かつ、セッションが維持されていれば、通信も継続されるようである